In [1]:
import matplotlib.pyplot as plt
from sklearn import preprocessing
import random
import numpy as np
import pandas as pd
from sklearn import datasets, svm, cross_validation, tree, preprocessing, metrics
import sklearn.ensemble as ske

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
sonar= pd.read_csv("E:/FCI/year 4/second term/NN/labs/lab5/sonar.csv")

In [3]:
print(sonar.head())

   0.0200  0.0371  0.0428  0.0207  0.0954  0.0986  0.1539  0.1601  0.3109  \
0  0.0453  0.0523  0.0843  0.0689  0.1183  0.2583  0.2156  0.3481  0.3337   
1  0.0262  0.0582  0.1099  0.1083  0.0974  0.2280  0.2431  0.3771  0.5598   
2  0.0100  0.0171  0.0623  0.0205  0.0205  0.0368  0.1098  0.1276  0.0598   
3  0.0762  0.0666  0.0481  0.0394  0.0590  0.0649  0.1209  0.2467  0.3564   
4  0.0286  0.0453  0.0277  0.0174  0.0384  0.0990  0.1201  0.1833  0.2105   

   0.2111 ...  0.0027  0.0065  0.0159  0.0072  0.0167  0.0180  0.0084  0.0090  \
0  0.2872 ...  0.0084  0.0089  0.0048  0.0094  0.0191  0.0140  0.0049  0.0052   
1  0.6194 ...  0.0232  0.0166  0.0095  0.0180  0.0244  0.0316  0.0164  0.0095   
2  0.1264 ...  0.0121  0.0036  0.0150  0.0085  0.0073  0.0050  0.0044  0.0040   
3  0.4459 ...  0.0031  0.0054  0.0105  0.0110  0.0015  0.0072  0.0048  0.0107   
4  0.3039 ...  0.0045  0.0014  0.0038  0.0013  0.0089  0.0057  0.0027  0.0051   

   0.0032  R  
0  0.0044  R  
1  0.0078  R  
2  0.

In [4]:
def splitDataSet(df):
    X = df.drop(df.columns[[60]], axis=1).values
    Y = df[df.columns[[60]]].values
    train_set_x_orig,test_set_x_orig ,train_set_y_orig,  test_set_y_orig = cross_validation.train_test_split(X,Y,test_size=0.2)
    print("train set x dim= "+str(train_set_x_orig.shape))
    print("train set y dim= "+str(train_set_y_orig.shape))
    print("test set x dim= "+str(test_set_x_orig.shape))
    print("test set y dim= "+str(test_set_y_orig.shape))
    print("after transpose ...")
    print("train set x dim= "+str(train_set_x_orig.T.shape))
    print("train set y dim= "+str(train_set_y_orig.shape))
    print("test set x dim= "+str(test_set_x_orig.T.shape))
    print("test set y dim= "+str(test_set_y_orig.shape))
    return train_set_x_orig.T,test_set_x_orig.T ,train_set_y_orig,  test_set_y_orig

In [5]:
train_set_x, test_set_x, train_set_y, test_set_y = splitDataSet(sonar)

train set x dim= (165, 60)
train set y dim= (165, 1)
test set x dim= (42, 60)
test set y dim= (42, 1)
after transpose ...
train set x dim= (60, 165)
train set y dim= (165, 1)
test set x dim= (60, 42)
test set y dim= (42, 1)


In [6]:
def initialize_with_zeros(dim):
    """
    This function creates a vector of zeros of shape (dim, 1) for w and initializes b to 0.
    
    Argument:
    dim -- number of parameters 
    
    Returns:
    w -- initialized vector of shape (dim, 1)
    b -- initialized scalar (corresponds to the bias)
    """
    
    w1 = np.zeros((dim, 1))
    b1 = 0.0
    w2 = np.zeros((dim, 1))
    b2 = 0.0
    return w1,w1, b1,b2

In [7]:
def Tanh(x):
   t = ( np.exp(x)-np.exp(-x))/(np.exp(x)+np.exp(-x))
   return t

In [8]:
def sigmoid(input):
    """
    Compute the sigmoid of input

    Arguments:
    input -- A scalar or numpy array of any size.

    Return:
    s -- sigmoid(input)
    """

    s = 1/(1 + np.exp(-input))
    
    return s

In [10]:
# GRADED FUNCTION: propagate

def propagate(w1, w2, b1, b2, X, Y):
    """
    Implement the cost function and its gradient 

    Arguments:
    w -- weights, a numpy array of size (num_Critirea, 1)
    b -- bias, a scalar
    X -- data of size (num_Critirea, number of examples)
    Y -- true "label" vector (containing 0 if non-cat, 1 if cat) of size (1, number of examples)

    Return:
    cost -- negative log-likelihood cost for logistic regression
    dw -- gradient of the loss with respect to w, thus same shape as w
    db -- gradient of the loss with respect to b, thus same shape as b
    
    """
    
    m = X.shape[1]
    
    # FORWARD PROPAGATION 
    z1=np.dot(w1.T, X) + b1
    
    A1 =Tanh(z1) 
    
    z2=np.dot(w2.T, A1) + b2
    
    A =sigmoid(z2) 
                
                
    # compute activation
    cost = - 1/m * np.sum((Y * np.log(A) + (1 - Y) * np.log(1 - A)))          # compute cost
    
    
    # BACKWARD PROPAGATION (TO FIND GRAD)
    dz2=A-Y
    
    dw2= 1/m * np.dot(dz2, A1.T)
    
    db2= 1/m * np.sum(dz2,axis=1,keepdims=True)
    
    dz1=np.dot(w2.T,dz2)*np.dot(A1,z1)
    
    dw1 = 1/m * np.dot(dz1, X.T)
    
    db1 =1/m * np.sum(dz1,axis=1,keepdims=True)
    
    
    grads = {"dw1": dw1,
             "db1": db1,
             "dz1": dz1,
             "dw2": dw2,
             "dz2": dz2,
             "db2": db2}
    
    return grads, cost